In [1]:
from keras.layers import Conv1D, Dense, MaxPool1D, concatenate, Flatten
from keras import Input, Model
from keras.layers.normalization import BatchNormalization
import numpy as np

Using TensorFlow backend.


In [36]:
data = pd.read_csv("data/train.csv")

In [37]:
data

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id
0,4,2019-07-16 00:00:00+08:00,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0,0
1,4,2019-07-16 00:00:00+08:00,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1
2,6,2019-07-16 00:00:00+08:00,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0,2
3,1,2019-07-16 00:00:00+08:00,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0,3
4,6,2019-07-16 00:00:00+08:00,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73534,6,2019-09-02 00:00:00+08:00,127613,39,24,36,279,0,1,1,0,0,0,0,0,0,0,73534
73535,2,2019-09-02 00:00:00+08:00,127620,38,46,10,51,0,0,1,0,0,0,0,0,0,0,73535
73536,2,2019-09-02 00:00:00+08:00,127696,32,Never open,Never login,Never checkout,0,0,0,0,0,0,0,0,0,0,73536
73537,2,2019-09-02 00:00:00+08:00,127807,38,5,34,Never checkout,2,4,4,0,0,0,0,0,0,1,73537


In [14]:
def multi_input_model():
    """構建多輸入模型"""
    country = Input(shape=(128, 1), name='country')
    
    grass_date = Input(shape=(128, 1), name='grass_date')
    
    subject_line_length = Input(shape=(128, 1), name='subject_line_length')
    
    last_open_day = Input(shape=(128, 1), name='last_open_day')
    
    last_login_day = Input(shape=(128, 1), name='last_login_day')
    
    last_checkout_day = Input(shape=(128, 1), name='last_checkout_day')
    
    open_count_last_10_days = Input(shape=(128, 1), name='open_count_last_10_days')
    open_count_last_30_days = Input(shape=(128, 1), name='open_count_last_30_days')
    open_count_last_60_days = Input(shape=(128, 1), name='open_count_last_60_days')
    
    login_count_last_10_days = Input(shape=(128, 1), name='login_count_last_10_days')
    login_count_last_30_days = Input(shape=(128, 1), name='login_count_last_30_days')
    login_count_last_60_days = Input(shape=(128, 1), name='login_count_last_60_days')

    checkout_count_last_10_days = Input(shape=(128, 1), name='checkout_count_last_10_days')
    checkout_count_last_30_days = Input(shape=(128, 1), name='checkout_count_last_30_days')
    checkout_count_last_60_days = Input(shape=(128, 1), name='checkout_count_last_60_days')
    
    c1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(country)
    c1 = MaxPool1D(pool_size=3)(c1)
    c2 = Conv1D(256, kernel_size=4, strides=1, activation='relu', padding='same')(c1)
    c2 = MaxPool1D(pool_size=3)(c2)
    c3 = Conv1D(512, kernel_size=2, strides=1, activation='relu', padding='same')(c2)
    c3 = MaxPool1D(pool_size=3)(c3)

    g1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(grass_date)
    g1 = MaxPool1D(pool_size=3)(g1)
    g2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(g1)
    g2 = MaxPool1D(pool_size=3)(g2)
    g3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(g2)
    g3 = MaxPool1D(pool_size=3)(g3)
     
    s1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(subject_line_length)
    s1 = MaxPool1D(pool_size=3)(s1)
    s2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(s1)
    s2 = MaxPool1D(pool_size=3)(s2)
    s3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(s2)
    s3 = MaxPool1D(pool_size=3)(s3)
    
    lo1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(last_open_day)
    lo1 = MaxPool1D(pool_size=3)(lo1)
    lo2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(lo1)
    lo2 = MaxPool1D(pool_size=3)(lo2)
    lo3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(lo2)
    lo3 = MaxPool1D(pool_size=3)(lo3)
    
    ll1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(last_login_day)
    ll1 = MaxPool1D(pool_size=3)(ll1)
    ll2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(ll1)
    ll2 = MaxPool1D(pool_size=3)(ll2)
    ll3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(ll2)
    ll3 = MaxPool1D(pool_size=3)(ll3)
    
    lc1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(last_checkout_day)
    lc1 = MaxPool1D(pool_size=3)(lc1)
    lc2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(lc1)
    lc2 = MaxPool1D(pool_size=3)(lc2)
    lc3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(lc2)
    lc3 = MaxPool1D(pool_size=3)(lc3)
    
    o10_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(open_count_last_10_days)
    o10_1 = MaxPool1D(pool_size=3)(o10_1)
    o10_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(o10_1)
    o10_2 = MaxPool1D(pool_size=3)(o10_2)
    o10_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(o10_2)
    o10_3 = MaxPool1D(pool_size=3)(o10_3)
    
    o30_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(open_count_last_30_days)
    o30_1 = MaxPool1D(pool_size=3)(o30_1)
    o30_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(o30_1)
    o30_2 = MaxPool1D(pool_size=3)(o30_2)
    o30_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(o30_2)
    o30_3 = MaxPool1D(pool_size=3)(o30_3)
    
    o60_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(open_count_last_60_days)
    o60_1 = MaxPool1D(pool_size=3)(o60_1)
    o60_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(o60_1)
    o60_2 = MaxPool1D(pool_size=3)(o60_2)
    o60_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(o60_2)
    o60_3 = MaxPool1D(pool_size=3)(o60_3)
    
    l10_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(login_count_last_10_days)
    l10_1 = MaxPool1D(pool_size=3)(l10_1)
    l10_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(l10_1)
    l10_2 = MaxPool1D(pool_size=3)(l10_2)
    l10_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(l10_2)
    l10_3 = MaxPool1D(pool_size=3)(l10_3)
    
    l30_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(login_count_last_30_days)
    l30_1 = MaxPool1D(pool_size=3)(l30_1)
    l30_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(l30_1)
    l30_2 = MaxPool1D(pool_size=3)(l30_2)
    l30_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(l30_2)
    l30_3 = MaxPool1D(pool_size=3)(l30_3)
    
    l60_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(login_count_last_60_days)
    l60_1 = MaxPool1D(pool_size=3)(l60_1)
    l60_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(l60_1)
    l60_2 = MaxPool1D(pool_size=3)(l60_2)
    l60_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(l60_2)
    l60_3 = MaxPool1D(pool_size=3)(l60_3)
    
    cc10_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(checkout_count_last_10_days)
    cc10_1 = MaxPool1D(pool_size=3)(cc10_1)
    cc10_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(cc10_1)
    cc10_2 = MaxPool1D(pool_size=3)(cc10_2)
    cc10_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(cc10_2)
    cc10_3 = MaxPool1D(pool_size=3)(cc10_3)
    
    cc30_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(checkout_count_last_30_days)
    cc30_1 = MaxPool1D(pool_size=3)(cc30_1)
    cc30_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(cc30_1)
    cc30_2 = MaxPool1D(pool_size=3)(cc30_2)
    cc30_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(cc30_2)
    cc30_3 = MaxPool1D(pool_size=3)(cc30_3)
    
    cc60_1 = Conv1D(128, kernel_size=8, strides=1, activation='relu', padding='same')(checkout_count_last_60_days)
    cc60_1 = MaxPool1D(pool_size=3)(cc60_1)
    cc60_2 = Conv1D(256, kernel_size=8, strides=1, activation='relu', padding='same')(cc60_1)
    cc60_2 = MaxPool1D(pool_size=3)(cc60_2)
    cc60_3 = Conv1D(512, kernel_size=8, strides=1, activation='relu', padding='same')(cc60_2)
    cc60_3 = MaxPool1D(pool_size=3)(cc60_3)
    
    x = concatenate([c3,g3,s3,lo3,ll3,lc3,o10_3,o30_3,o60_3,l10_3,l30_3,l60_3,cc10_3,cc30_3,cc60_3])
    x = Flatten()(x)

    x = Dense(256, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    
    open_flag = Dense(2, activation='sigmoid', name='open_flag')(x)

    model = Model(inputs=[country,
                          grass_date,
                          subject_line_length,
                          last_open_day,
                          last_login_day,
                          last_checkout_day,
                          open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,
                          login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,
                          checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days],
                  outputs=[open_flag])
    model.summary()

    return model

In [15]:
model = multi_input_model()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
country (InputLayer)            (None, 128, 1)       0                                            
__________________________________________________________________________________________________
grass_date (InputLayer)         (None, 128, 1)       0                                            
__________________________________________________________________________________________________
subject_line_length (InputLayer (None, 128, 1)       0                                            
__________________________________________________________________________________________________
last_open_day (InputLayer)      (None, 128, 1)       0                                            
____________________________________________________________________________________________